In [134]:
def get_stock_info(stock_name, start = 0, end = 0):
    import pandas as pd
    import datetime
    import pandas_datareader.data as web
    import fix_yahoo_finance as yf
    end = datetime.date.today()
    start = end - datetime.timedelta(days = 365*10)
    try:
        data = web.get_data_yahoo(stock_name, start,end)
        data['Return'] = data['Close'].pct_change()
        data = data.dropna()
        std = data['Return'].describe().loc['std']
        price = data['Close'].iloc[len(data)-1]
        
        return price, std
        
    except:
        print('Cannot find stock data')
    
    

In [143]:
get_stock_info('AAPL')

(176.77999877929688, 0.01690404738972001)

In [179]:
def get_rf(date = 0, tenor = '10 yr'):
    import requests
    from bs4 import BeautifulSoup
    import datetime
    url = 'https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx?data=yield'
    response = requests.get(url)

    if response.status_code == 200:
        #print("Success")
        data_page = BeautifulSoup(response.content,'lxml')
        yield_table = data_page.find('table', class_ = 't-chart')
        trs = yield_table.find_all('tr')
        col = [trs[0].find_all('th', scope = 'col')[i].get_text() 
                for i in range(1,len(trs[0].find_all('th', scope = 'col')))]
        index = [trs[i].find_all('td', scope = 'row')[0].get_text() for i in range(1,len(trs))]

        # first commit

        rates = [[float(trs[i].find_all('td')[j].get_text()) for j in range(1,len(col)+1)] for i in range(1,len(trs))]
        data = pd.DataFrame(rates, columns = col, index = index)
        data = data/100 #pct_value
        
        date = data.index.values[len(data)-1] #latest data

        rf_rate = data.loc[date][tenor]
    else:
        print('Failure')
    return rf_rate

In [178]:
get_rf()

0.030600000000000002